In [5]:

import numpy as np
import json
import math

from imutils.video import VideoStream
import argparse
import datetime
import motmetrics as mm
import imutils
import time
from sklearn import preprocessing
import cv2 as cv
import time
from sort import *

import json
from utils import load_detections

from detection import Detection
from tracker import Tracker

acc = mm.MOTAccumulator(auto_id=True)

iou_overlaps = []
desc_dists = []
confusion_frames = []
confusion_tracks = []
confusion_distances =[]
colors = [[0,0,128],[0,255,0],[0,0,255],[255,0,0],[0,128,128],[128,0,128],[128,128,0],[255,255,0],[0,255,255],[255,255,0],[128,0,0],[0,128,0]
         ,[0,128,255],[0,255,128],[255,0,128],[128,255,0],[255,128,0],[128,255,255],[128,0,255],[128,128,128],[128,255,128]]
tracking_methods=['kalman_corners']
#tracking_methods=['center_flow','keypoint_flow','kalman_center','kalman_corners','SORT']
detectors = ['yolo']
#detectors = ['ssd300','retinanet','yolo']
#'center_fow','keypoint_flow','kalman_center','kalman_corners',
datasets=['garda_2']
times = {}
for dataset in datasets:
    times[dataset]={}
    images_input_path='../%s/'%dataset
    image_id_prefix= dataset
    frame_width=1032
    frame_height=778
    if(dataset=='venc'):
        frame_width = 1280
        frame_height = 960
    if(dataset=='modd'):
        frame_width=640
        frame_height=464
    if(dataset=='garda_1' or dataset=='garda_2'):
        frame_width=1280
        frame_height=720
    if(dataset=='mot_1'):
        frame_width=768
        frame_height=576
    iou_threshold = 0.1
    for detector in detectors:
        times[dataset][detector] = {}
        boat_class=8
        min_conf=0
        if(detector=='ssd300'):
            boat_class=4
            min_conf=0
        if(detector=='def'):
            boat_class=1
        

        path = '%s/%s_videos'%(detector,image_id_prefix)
        detections = load_detections(image_id_prefix,detector,boat_class,0)
        
        for tracking_method in tracking_methods:
            times[dataset][detector][tracking_method] = []
            video_output_path='%s/%s.avi'%(path,tracking_method)
            json_output_path='%s/%s.json'%(path,tracking_method)
            out_tracking = cv.VideoWriter(video_output_path,cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
            frameCount =0
            no_tracking_res = [] 
            tracking_res = []
            kalman_trackers=[]
            # initialize the first frame in the video stream
            frameCount =0
            step_up = 0.1
            step_down = 0.2
            print('Running: Dataset:%s, Detector:%s, Tracker:%s, @%dx%d'%(dataset,detector,tracking_method,frame_width,frame_height))
            preds = []
            tracks=[]
            started = False
            multiplier=0
            cc=0
            prev_frame=None
            
            total_frames=641
            if(tracking_method=='SORT'):
                mot_tracker = Sort()
            else:
                tracker_wrapper = Tracker(tracking_method)
                tracker_wrapper.frame_width = frame_width
                tracker_wrapper.frame_height = frame_height
                if(dataset=='modd'):
                    tracker_wrapper.A = np.array([int(frame_width/2),int(frame_height/2)])
                    tracker_wrapper.B = np.array([int(frame_width/5),int(frame_height-1)])
                    tracker_wrapper.C= np.array([int(4*frame_width/5),frame_height-1])
                elif(dataset=='graal_1' or dataset == 'graal_2' or dataset=='garda_1' or dataset=='garda_2'):
                    tracker_wrapper.A = np.array([int(frame_width/2),int(frame_height/2)])
                    tracker_wrapper.B = np.array([int(frame_width/5),int(frame_height-1)])
                    tracker_wrapper.C= np.array([int(4*frame_width/5),frame_height-1])
            while frameCount<total_frames:
                
                # grab the current frame and initialize the occupied/unoccupied
                # text
                frame = cv.imread('%s%s.jpg'%(images_input_path,str(frameCount+1).zfill(5)))
                
                
                    
                
                
                # if the frame could not be grabbed, then we have reached the end
                # of the video
                if frame is None:
                    break

                if(frameCount<0):
                    continue

                preds = []
                if '%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5)) in detections and tracking_method=='SORT':

                    for box in detections['%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5))]:

                        if(box.conf<min_conf):

                            continue
                        
                        temp_pred = box[2:]

                        temp_pred = np.insert(temp_pred,4,box[1])

                        preds.append(temp_pred)
                        
                          
                
                start= time.time()
                if(tracking_method=='SORT'):

                    preds = np.asarray(preds)
                    trackers = mot_tracker.update(preds)
                    to_display = []
                    for itrk,tracker in enumerate(trackers):
                        to_display.append([tracker[4],boat_class,preds[itrk][4],tracker[0],tracker[1],tracker[2],tracker[3]])
                else:   
                    if '%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5)) in detections:
                        preds = detections['%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5))]
                    for det in preds:
                        det.calc_hog_descriptor(frame)
                    tracker_wrapper.track(preds,frame,prev_frame)
                    
                    
                    to_display = tracker_wrapper.get_display_tracks()
                    




                        #print(acc.mot_events.loc[frameId])
               
                col_points = []#tracker_wrapper.get_collision_points()
                if(len(col_points)>0):
                    col = [0,0,255]
                    cv.putText(frame,'Collision detected!', (20, 20),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    
                
                else:
                    col = [0,255,255]
                
                
               
                
                for p in col_points:
                    cv.circle(frame,(int(p[0]),int(p[1])),8,(0,0,255),1)
                i=0
                for box in to_display:
                    
                # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
                   
                    xmin = int(box.xmin)
                    ymin = int(box.ymin)
                    xmax =int(box.xmax)
                    ymax =int(box.ymax)
                    
                    cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), colors[int(box.track_id)%len(colors)], 3)
                    #cv.rectangle(frame, (int(box.new_box[0] - box.new_box[2]/2), int(box.new_box[1] - box.new_box[3]/2)), (int(box.new_box[0] + box.new_box[2]/2),int(box.new_box[1] + box.new_box[3]/2)), (255,255,255), 2)
                    p1 = box.center()
                    p2 = box.center()
                    p2[0] += box.offset[0]*2
                    p2[1] += box.offset[1]*2
                    cv.arrowedLine(frame,(int(p1[0]),int(p1[1])),(int(p2[0]),int(p2[1])),(0,0,255),2)
                    if(tracking_method=='kalman_center'):
                        
                        cv.circle(frame,(int(predictions[i][0][0]),int(predictions[i][1][0])),5,(255,0,0),2)
                    #if(tracking_method=='kalman_corners'):
                        #cv.circle(frame,(int(predictions[i][0][0]),int(predictions[i][1][0])),5,(255,0,0),2)
                        #cv.circle(frame,(int(predictions[i][2][0]),int(predictions[i][3][0])),5,(255,0,0),2)
                    cv.putText(frame,'{:.2f}'.format( box.conf), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    

                    tracking_res.append({"image_id" : frameCount+1, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : np.minimum(1.0,box.conf),"id":box.track_id})
                    #f.write("graal_2/%s.jpg,%s,%d,%f,%f,%f,%f,%f\n"%(str(frameCount+1).zfill(5),classes[int(box[1])],box[1],box[2],xmin,ymin,xmax,ymax))
                    i+=1
                    times[dataset][detector][tracking_method].append(time.time()-start)
                
                out_tracking.write(frame)
                #cv.imwrite('debug_frames/%s.jpg'%str(frameCount+1),frame)
                frameCount+=1
                prev_frame=frame
                flow = None
            # cleanup the camera and close any open windows

            out_tracking.release()
            
            


            with open(json_output_path, 'w') as outfile:  
                json.dump(tracking_res, outfile)
                
                

Running: Dataset:garda_2, Detector:yolo, Tracker:kalman_corners, @1280x720
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0

offset  [-0.294036865234375, -5.83343505859375]
offset  [0. 0.]
offset  [0. 0.]
offset  [-2.1097412109375, -6.9228515625]
offset  [-1.296600341796875, -5.6221923828125]
offset  [6.0926513671875, -6.823211669921875]
offset  [-0.7035369873046875, -7.1256103515625]
offset  [-2.28955078125, -5.736602783203125]
offset  [0. 0.]
offset  [0. 0.]
offset  [-0.4825439453125, -6.0711669921875]
offset  [3.976806640625, -5.624298095703125]
offset  [-0.0849456787109375, -7.1715087890625]
offset  [0. 0.]
offset  [0.60040283203125, -6.1563720703125]
offset  [-4.904754638671875, -5.277374267578125]
offset  [1.816650390625, -3.827178955078125]
offset  [0. 0.]
offset  [-0.276702880859375, -7.20953369140625]
offset  [-2.1646728515625, -7.68359375]
offset  [1.0849609375, -4.82879638671875]
offset  [0.032958984375, -12.20623779296875]
offset  [0. 0.]
offset  [0.040740966796875, -9.310272216796875]
offset  [-0.754150390625, -6.61529541015625]
offset  [0.4888153076171875, -15.01397705078125]
offset  [15.567749

offset  [-13.541473388671875, 1.93243408203125]
offset  [-12.060012817382812, 4.48162841796875]
offset  [0. 0.]
offset  [-11.60797119140625, -3.92694091796875]
offset  [0. 0.]
offset  [-14.053482055664062, 5.67718505859375]
offset  [-9.091217041015625, 2.29486083984375]
offset  [0. 0.]
offset  [0. 0.]
offset  [-6.66485595703125, -2.603546142578125]
offset  [-28.521255493164062, 25.92730712890625]
offset  [0. 0.]
offset  [-13.160003662109375, 4.0185546875]
offset  [-13.86224365234375, 2.931243896484375]
offset  [0. 0.]
offset  [-19.78350830078125, 23.186447143554688]
offset  [0. 0.]
offset  [-14.596588134765625, 4.959320068359375]
offset  [3.355133056640625, 2.650390625]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [127.23870849609375, 15.28173828125]
offset  [-12.643890380859375, 3.839385986328125]
offset  [0. 0.]
offset  [-18.756988525390625, 1.921356201171875]
offset  [-76.92193603515625, -11.8408203125]
offset  [0. 0.]
offset  [0. 0.]
offset  [-14.911148071289062, 4.10247

offset  [-7.236595153808594, 18.636260986328125]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [-3.40179443359375, -1.11407470703125]
offset  [0. 0.]
offset  [3.9873046875, -4.4185791015625]
offset  [-4.511463165283203, 22.501861572265625]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0.679290771484375, -1.235504150390625]
offset  [0. 0.]
offset  [-4.495452880859375, -9.827789306640625]
offset  [0. 0.]
offset  [-5.056194305419922, 20.679595947265625]
offset  [-5.8629608154296875, 2.624908447265625]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [-0.662261962890625, -0.851898193359375]
offset  [0. 0.]
offset  [-7.431671142578125, -3.79962158203125]
offset  [-5.6410980224609375, 4.231353759765625]
offset  [0. 0.]
offset  [-4.653408050537109, 14.00909423828125]
offset  [0. 0.]
offset  [0. 0.]
offset  [0. 0.]
offset  [-8.488189697265625, 5.226470947265625]
offset  [0. 0.]
offset  [-3.250896453857422, 6.67767333984375]
offset  [-

offset  [-13.4730224609375, -8.504852294921875]
offset  [34.115264892578125, 13.509063720703125]
offset  [0. 0.]
offset  [-1.6142938137054443, -2.138671875]
offset  [0. 0.]
offset  [-7.572601318359375, -7.972198486328125]
offset  [-1.819549560546875, -8.40234375]
offset  [-4.88906317949295, -4.25054931640625]
offset  [-11.275909423828125, -9.451873779296875]
offset  [0. 0.]
offset  [0. 0.]
offset  [-5.946220397949219, -2.06817626953125]
offset  [-6.1011505126953125, -15.49188232421875]
offset  [0. 0.]
offset  [-6.82000732421875, 0.460601806640625]
offset  [-3.53118896484375, -6.199615478515625]
offset  [-10.6146240234375, -16.591705322265625]
offset  [0. 0.]
offset  [-5.126517832279205, -5.993438720703125]
offset  [-10.1016845703125, -12.309738159179688]
offset  [0. 0.]
offset  [-21.08135986328125, -28.718719482421875]
offset  [0. 0.]
offset  [-20.475738525390625, -27.2113037109375]
offset  [-6.71856689453125, -20.484344482421875]
offset  [-9.637420654296875, -11.863113403320312]
offse

...

offset  [-2.2197341918945312, -1.0285186767578125]
offset  [-7.691036224365234, 6.2387542724609375]
offset  [-1.604400634765625, -17.56182861328125]
offset  [-1.1449661254882812, 1.2386627197265625]
offset  [-6.602725982666016, -29.82049560546875]
offset  [-2.611440658569336, -18.89874267578125]
offset  [-1.503326416015625, 1.909393310546875]
offset  [0.2920207977294922, 31.53033447265625]
offset  [0.195098876953125, -12.92041015625]
offset  [-2.0404281616210938, 3.4179840087890625]
offset  [-2.9049835205078125, 4.459564208984375]
offset  [-1.3204727172851562, 27.47308349609375]
offset  [-1.837554931640625, -5.09625244140625]
offset  [-6.43902587890625, 4.943328857421875]
offset  [-0.9126920700073242, 23.67193603515625]
offset  [2.680084228515625, -4.970672607421875]
offset  [-0.5012702941894531, 18.80487060546875]
offset  [-4.717582702636719, 8.620590209960938]
offset  [2.222412109375, -3.789215087890625]
offset  [-4.657752990722656, 7.490966796875]
offset  [-1.1967124938964844, 14.69

offset  [0.17760848999023438, 13.5989990234375]
offset  [2.166668653488159, 24.529052734375]
offset  [1.1700439453125, 3.5360107421875]
offset  [0.25347900390625, 13.100341796875]
offset  [8.668853759765625, 1.93756103515625]
offset  [-0.45385967195034027, -22.765548706054688]
offset  [0. 0.]
offset  [1.1684532165527344, 10.756561279296875]
offset  [0.0972888171672821, 40.465728759765625]
offset  [1.336883544921875, 4.67425537109375]
offset  [0. 0.]
offset  [0.23604202270507812, 7.363067626953125]
offset  [0.48309896886348724, 26.49493408203125]
offset  [0. 0.]
offset  [1.0691757202148438, 4.445037841796875]
offset  [1.180397242307663, 20.77081298828125]
offset  [0. 0.]
offset  [0.914459228515625, 4.814056396484375]
offset  [0.3207893371582031, 5.578125]
offset  [0.771942138671875, 1.487060546875]
offset  [0. 0.]
offset  [-0.9299888610839844, 5.478302001953125]
offset  [1.23748779296875, 22.6083984375]
offset  [-0.15078125894069672, 13.19903564453125]
offset  [0. 0.]
offset  [-0.048629

offset  [2.8836517333984375, -5.21142578125]
offset  [5.206634521484375, 0.717864990234375]
offset  [7.0490570068359375, -1.418121337890625]
offset  [4.7689971923828125, -0.380126953125]
offset  [3.0899658203125, 2.224456787109375]
offset  [15.24551773071289, 36.57319641113281]
offset  [0.04583740234375, 4.0526123046875]
offset  [0. 0.]
offset  [4.685033798217773, -0.6934356689453125]
offset  [7.63580322265625, -1.599609375]
offset  [4.446441650390625, 21.96319580078125]
offset  [30.917015075683594, 18.756195068359375]
offset  [0. 0.]
offset  [22.126413345336914, 64.37605285644531]
offset  [0.906707763671875, 17.9451904296875]
offset  [18.036888122558594, 16.978851318359375]
offset  [16.110092163085938, 36.48345947265625]
offset  [0. 0.]
offset  [2.367462158203125, 11.971435546875]
offset  [5.11883544921875, 8.939788818359375]
offset  [0. 0.]
offset  [11.253562927246094, 13.114990234375]
offset  [12.206295013427734, 22.70611572265625]
offset  [5.591278076171875, 4.121917724609375]
offs

offset  [-0.332489013671875, -17.807373046875]
offset  [1.2880096435546875, -18.5147705078125]
offset  [0. 0.]
offset  [-1.9149017333984375, -18.85504150390625]
offset  [2.8820343017578125, -20.2889404296875]
offset  [-0.392974853515625, -17.302734375]
offset  [0. 0.]
offset  [-1.0268096923828125, -23.50238037109375]
offset  [-0.614349365234375, -14.586563110351562]
offset  [3.0081329345703125, -16.1597900390625]
offset  [0.5517959594726562, -15.438812255859375]
offset  [3.3831329345703125, -8.375152587890625]
offset  [2.5874786376953125, -23.049224853515625]
offset  [6.640594482421875, -7.6324462890625]
offset  [0. 0.]
offset  [3.777099609375, -13.504165649414062]
offset  [0. 0.]
offset  [3.07513427734375, -2.5943603515625]
offset  [0. 0.]
offset  [3.1738357543945312, -7.103546142578125]
offset  [9.441543579101562, 3.697357177734375]
offset  [0. 0.]
offset  [0. 0.]
offset  [7.707427978515625, 15.584014892578125]
offset  [6.8447113037109375, -0.9122161865234375]
offset  [0. 0.]
offset 

offset  [1.303314208984375, -8.86614990234375]
offset  [2.066680908203125, -7.74359130859375]
offset  [0. 0.]
offset  [8.700836181640625, -13.127456665039062]
offset  [0. 0.]
offset  [9.196319580078125, -5.7921142578125]
offset  [3.2508544921875, -6.814239501953125]
offset  [0. 0.]
offset  [0. 0.]
offset  [6.1155242919921875, -11.089019775390625]
offset  [6.316925048828125, -2.57708740234375]
offset  [0. 0.]
offset  [7.393890380859375, -3.515594482421875]
offset  [6.06915283203125, -6.527252197265625]
offset  [6.771575927734375, -0.5142822265625]
offset  [6.4854736328125, -0.1419677734375]
offset  [5.0911712646484375, -1.7720489501953125]
offset  [4.461456298828125, 1.49774169921875]
offset  [0. 0.]
offset  [6.93780517578125, 2.02081298828125]
offset  [6.358184814453125, 2.75286865234375]
offset  [0. 0.]
offset  [3.76239013671875, 6.548583984375]
offset  [5.49365234375, 5.2011566162109375]
offset  [7.10394287109375, 11.923583984375]
offset  [0. 0.]
offset  [4.242828369140625, 6.7037963

In [6]:
print(np.average(times['garda_2']['yolo']['kalman_corners']))

0.04514923484077281
